search for 5+% losses. analyse the behavior prior and after

In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np

In [ ]:
dax = ["ADS.DE", "AIR.PA", "ALV.DE", "BAS.DE", "BAYN.DE", "BEI.DE", "BMW.DE", "BNR.DE", "CBK.DE", "CON.DE", "1COV.DE", "DTG.DE", "DBK.DE", "DB1.DE", "DHL.DE", "DTE.DE", "EOAN.DE", "FRE.DE", "FME.DE", "HNR1.DE", "HEI.DE", "HEN3.DE", "IFX.DE", "MBG.DE", "MRK.DE", "MTX.DE", "MUV2.DE", "PAH3.DE", "P911.DE", "QIA.DE", "RHM.DE", "RWE.DE", "SAP.DE", "SRT3.DE", "SIE.DE", "ENR.DE", "SHL.DE", "SY1.DE", "VOW3.DE", "VNA.DE", "ZAL.DE"]

In [ ]:
# Full implementation method (recommended):
import pandas as pd
def get_sp500_tickers():
    table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    df = table[0]
    return df['Symbol'].tolist()

# Usage:
sp500_tickers = get_sp500_tickers()


In [ ]:
# data = yf.download(sp500_tickers, start="2015-01-01", end="2025-05-01", interval="1d")
data = yf.download(dax, start="2015-01-01", end="2025-05-01", interval="1d")

In [ ]:
df_SPY = yf.download("SPY", start="2015-01-01", end="2025-05-01", interval="1d")

In [ ]:
df_SPY = df_SPY.stack(level=1).rename_axis(['Date', 'Ticker']).reset_index()


In [ ]:
df_SPY

In [ ]:
df = data.stack(level=1).rename_axis(['Date', 'Ticker']).reset_index()


In [ ]:
# Sort to ensure chronological order
df = df.sort_values(by=['Ticker', 'Date'])

# Group by ticker and calculate % changes
df['Prev_Close'] = df.groupby('Ticker')['Close'].shift(1)

df['Day_Pct'] = df['Close'] / df['Open'] - 1
df['Night_Pct'] = df['Open'] / df['Prev_Close'] - 1

# Identify where either drop exceeds -5%
drop_mask = (df['Day_Pct'] <= -0.1) | (df['Night_Pct'] <= -0.1)
drop_points = df[drop_mask]


In [ ]:
# Sort to ensure chronological order
df_SPY = df_SPY.sort_values(by=['Ticker', 'Date'])

# Group by ticker and calculate % changes
df_SPY['Prev_Close'] = df_SPY.groupby('Ticker')['Close'].shift(1)

df_SPY['Day_Pct'] = df_SPY['Close'] / df_SPY['Open'] - 1
df_SPY['Night_Pct'] = df_SPY['Open'] / df_SPY['Prev_Close'] - 1

# Identify where either drop exceeds -5%
drop_mask_SPY = (df_SPY['Day_Pct'] <= -0.05) | (df_SPY['Night_Pct'] <= -0.05)
drop_points_SPY = df_SPY[drop_mask_SPY]

In [ ]:
drop_points_filter = drop_points[~drop_points["Date"].isin(drop_points_SPY["Date"])]

In [ ]:
# # Calculate time difference between current and previous row within same ID
# drop_points_filter['Date_Diff'] = drop_points_filter.groupby('Ticker')['Date'].diff()

# # Mark entries where the date difference is less than 30 days
# drop_points_filter['Remove'] = drop_points_filter['Date_Diff'].dt.days < 30

# # Fill NA (first entry per group) with False
# drop_points_filter['Remove'] = drop_points_filter['Remove'].fillna(False)

# # Keep only rows not marked for removal
# drop_points_filter = drop_points_filter[~drop_points_filter['Remove']].drop(columns=['Date_Diff', 'Remove'])

In [ ]:

# Final results container
results = []

# Loop over each drop
for _, row in drop_points_filter.iterrows():
    ticker = row['Ticker']
    drop_date = row['Date']
    drop_pct = min(row['Day_Pct'], row['Night_Pct'])

    # Define windows
    start_date_before3 = drop_date - pd.DateOffset(months=3)
    start_date_after3 = drop_date + pd.DateOffset(months=3)
    
    start_date_before2 = drop_date - pd.DateOffset(months=2)
    start_date_after2 = drop_date + pd.DateOffset(months=2)
    
    start_date_before1 = drop_date - pd.DateOffset(months=1)
    start_date_after1 = drop_date + pd.DateOffset(months=1)

    day_before_drop = drop_date - pd.DateOffset(days=1)
    day_after_drop = drop_date + pd.DateOffset(days=1)
    
    # Extract price 3 months before and after
    ticker_df = df[df['Ticker'] == ticker].set_index('Date')

    try:
        three_months = {}
        for start_date_before, start_date_after, months in [(start_date_before3, start_date_after3, 3), (start_date_before2, start_date_after2, 2), (start_date_before1, start_date_after1, 1)]:
            price_before = ticker_df.loc[start_date_before:, 'Close'].iloc[0]
            price_before_drop_day = (ticker_df.loc[day_before_drop:, 'Close'].iloc[0] + ticker_df.loc[day_before_drop:, 'Open'].iloc[0]) / 2
            price_after_drop_day = (ticker_df.loc[day_after_drop:, 'Close'].iloc[0] + ticker_df.loc[day_after_drop:, 'Open'].iloc[0]) / 2
            price_after = ticker_df.loc[start_date_after:, 'Close'].iloc[0]

            pct_change_before = (price_before_drop_day / price_before) - 1
            pct_change_after = (price_after / price_after_drop_day) - 1
            three_months[f"Pct_Change_Before_{months}M"] = pct_change_before
            three_months[f"Pct_Change_After_{months}M"] = pct_change_after

        results.append({
            'Ticker': ticker,
            'Drop_Date': drop_date,
            'Drop_Percentage': drop_pct,
            **three_months
        })
    except IndexError:
        # If before/after data is missing (e.g., start/end of dataset)
        continue

# Create final DataFrame
final_df = pd.DataFrame(results)

In [ ]:
final_df

In [ ]:
from datetime import datetime, timedelta

In [ ]:
# sell point detection: 
#   A: drop of 10%
#   B: momentum shifts
lookback_period_long = 90
lookback_period_short = 30

# get start date
def invest_journey(row):
    df_ticker = df[(df["Ticker"] == row["Ticker"]) & (df["Date"] > row["Drop_Date"])].copy()
    df_ticker.reset_index(drop=True, inplace=True)
    buy_price = df_ticker.head(1).High.iloc[0]
    buy_date = df_ticker.head(1).Date.iloc[0]
    df_ticker["momentum_long"] = df_ticker['Close'].rolling(window=lookback_period_long).mean()
    df_ticker["momentum_short"] = df_ticker['Close'].rolling(window=lookback_period_short).mean()
    df_ticker["gain"] = df_ticker["Close"].pct_change(periods=1).cumsum()
    df_ticker["sell"] = np.where((df_ticker["momentum_long"] > df_ticker["momentum_short"]), 1, 0) #(df_ticker["gain"] < -0.15) |  | (df_ticker["Day_Pct"] < -0.1) | (df_ticker["Night_Pct"] < -0.1) 
    if df_ticker[df_ticker["sell"] == 1].shape[0] == 0:
        sell_price = df_ticker.iloc[[-1]].Low.iloc[0]
        sell_date = df_ticker.iloc[[-1]].Date.iloc[0]      
        # manipulate for percentages
        df_ticker.iloc[[-1]]["Close"] = sell_price
        df_ticker.loc[0, 'Close'] = buy_price
        df_ticker['Pct_Change'] = df_ticker['Close'].pct_change(periods=1)
    else:    
        sell_price = df_ticker.iloc[df_ticker[df_ticker["Date"] == (df_ticker.loc[df_ticker["sell"] == 1, "Date"].min())].index + 1].Low.iloc[0]
        sell_date = df_ticker.iloc[df_ticker[df_ticker["Date"] == (df_ticker.loc[df_ticker["sell"] == 1, "Date"].min())].index + 1].Date.iloc[0]
        # manipulate for percentages
        df_ticker.loc[df_ticker[df_ticker["Date"] == (df_ticker.loc[df_ticker["sell"] == 1, "Date"].min())].index + 1, "Close"] = sell_price
        df_ticker.loc[0, 'Close'] = buy_price
        df_ticker['Pct_Change'] = df_ticker['Close'].pct_change(periods=1)
    return [row["Ticker"], row["Drop_Date"], buy_price, sell_price, sell_price / buy_price - 1, buy_date, sell_date]

In [ ]:
results = []
for _, row in final_df.iterrows():
    results.append(invest_journey(row))

In [ ]:
results_df = pd.DataFrame(results, columns=["Ticker", "Drop_Date", "Buy_Price", "Sell_Price", "Pct_Change", "Buy Date", "Sell Date"])

In [ ]:
results_df["Pct_Change"].mean()

In [ ]:
results_df

In [ ]:
df_long = pd.DataFrame({
    'id': pd.concat([results_df['Ticker'], results_df['Ticker']]).values,
    'date': pd.concat([results_df['Buy Date'], results_df['Sell Date']]).values,
    'value': pd.concat([pd.Series([pd.NA] * len(results_df)), results_df['Pct_Change']]).values, 
    'category': ['Buy'] * len(results_df) + ['Sell'] * len(results_df)
})

In [ ]:
df_long.sort_values("date")

In [ ]:
portfolio = 200

for row in df_long.iterrows():
    if row[1]["category"] == "Buy":
        portfolio += row[1]["value"] * portfolio
    else:
        portfolio += row[1]["value"] * portfolio

In [ ]:
import numpy as np

# Define bins from -1.0 to 1.0 in steps of 0.1
bins = np.arange(-1.0, 1.1, 0.1)

# Create labels for each bin
labels = [f"{round(bins[i],1)} to {round(bins[i+1],1)}" for i in range(len(bins)-1)]

# Bin the data
for m in ["3M", "2M", "1M"]:
    final_df[f'Pct_After_Bin_{m}'] = pd.cut(final_df[f'Pct_Change_After_{m}'], bins=bins, labels=labels, include_lowest=True)


In [ ]:
# Count occurrences in each bin
histogram = final_df['Pct_After_Bin_1M'].value_counts().sort_index().reset_index()
histogram.columns = ['Pct_Change_Range', 'Count_1M']

h2M = final_df['Pct_After_Bin_2M'].value_counts().sort_index().reset_index()
h2M.columns = ['Pct_Change_Range', 'Count_2M']

h3M = final_df['Pct_After_Bin_3M'].value_counts().sort_index().reset_index()
h3M.columns = ['Pct_Change_Range', 'Count_3M']

histogram = pd.merge(histogram, h2M, on='Pct_Change_Range')
histogram = pd.merge(histogram, h3M, on='Pct_Change_Range')


In [ ]:
histogram["Count_1M"] = round((histogram["Count_1M"] / histogram["Count_1M"].sum()) * 100, 2)
histogram["Count_2M"] = round((histogram["Count_2M"] / histogram["Count_2M"].sum()) * 100, 2)
histogram["Count_3M"] = round((histogram["Count_3M"] / histogram["Count_3M"].sum()) * 100, 2)

In [ ]:
histogram